In [40]:
import pandas as pd
from collections import deque, defaultdict
import heapq
import time
from concurrent.futures import ProcessPoolExecutor, as_completed

In [41]:
path_candidates='/home/it/Desktop/RUN/Official/Source/CSAB_SuperNumerary/filtered_candidates.csv'
path_seats='/home/it/Desktop/RUN/Official/Source/CSAB_SuperNumerary/Seats.csv'
# path_seatsFor='/home/it/Desktop/RUN/Official/Source/CSAB_SuperNumerary/Seats_Foreign.csv'
path_normChoices='/home/it/Desktop/RUN/Official/CSAB_SuperNumerary/Normal_Virtualized_Choices.csv'
path_choices='/home/it/Desktop/RUN/Official/Source/CSAB_SuperNumerary/Choice.csv'
path_allotment='/home/it/Desktop/RUN/Official/Source/CSAB_SuperNumerary/Allotment_NIC.csv'
path_output='allotment_NITK.csv'
path_error='allotment_NITK_unalloted.csv'

In [42]:
class Program:
    def __init__(self, inst_code, brcode, quota, cat, subcat, gender, capacity, min_cutoff=0, round_no=1):
        self.inst_code = inst_code
        self.brcode = brcode
        self.capacity = capacity
        self.gender = gender
        self.quota = quota
        self.cat = cat
        self.subcat = subcat
        self.min_cutoff = min_cutoff
        self.round_no = round_no
        self.waitlist = []

    def is_eligible(self, rk):
        if self.round_no == 1:
            return True
        return rk <= self.min_cutoff

    def add_candidate(self, rk, roll_no):
        if rk>0:
            heapq.heappush(self.waitlist, (-rk, roll_no))
            return True
        return False

    def remove_least_preferred(self):
        return heapq.heappop(self.waitlist)

class Candidate:
    def __init__(self, roll_no, 
                 AI_Eng_CRL_Rank, AI_Eng_OBC_NCL_Rank, AI_Eng_SC_Rank, AI_Eng_ST_Rank, AI_Eng_EWS_Rank, 
                 AI_Eng_CRL_PD_Rank, AI_Eng_OBC_NCL_PD_Rank, AI_Eng_SC_PD_Rank, AI_Eng_ST_PD_Rank, AI_Eng_EWS_PD_Rank, 
                 AI_ARC_CRL_Rank, AI_ARC_OBC_NCL_Rank, AI_ARC_SC_Rank, AI_ARC_ST_Rank, AI_ARC_EWS_Rank, 
                 AI_ARC_CRL_PD_Rank, AI_ARC_OBC_NCL_PD_Rank, AI_ARC_SC_PD_Rank, AI_ARC_ST_PD_Rank, AI_ARC_EWS_PD_Rank, 
                 AI_Pln_CRL_Rank, AI_Pln_OBC_NCL_Rank, AI_Pln_SC_Rank, AI_Pln_ST_Rank, AI_Pln_EWS_Rank, 
                 AI_Pln_CRL_PD_Rank, AI_Pln_OBC_NCL_PD_Rank, AI_Pln_SC_PD_Rank, AI_Pln_ST_PD_Rank, AI_Pln_EWS_PD_Rank):

        self.roll_no = roll_no
        self.current_index = 1
        self.max_index = 0
        self.choice_list = {}
        self.choice_list_mapper = {}

        self.AI_Eng_CRL_Rank = AI_Eng_CRL_Rank
        self.AI_Eng_OBC_NCL_Rank = AI_Eng_OBC_NCL_Rank
        self.AI_Eng_SC_Rank = AI_Eng_SC_Rank
        self.AI_Eng_ST_Rank = AI_Eng_ST_Rank
        self.AI_Eng_EWS_Rank = AI_Eng_EWS_Rank
        self.AI_Eng_CRL_PD_Rank = AI_Eng_CRL_PD_Rank
        self.AI_Eng_OBC_NCL_PD_Rank = AI_Eng_OBC_NCL_PD_Rank
        self.AI_Eng_SC_PD_Rank = AI_Eng_SC_PD_Rank
        self.AI_Eng_ST_PD_Rank = AI_Eng_ST_PD_Rank
        self.AI_Eng_EWS_PD_Rank = AI_Eng_EWS_PD_Rank
        
        self.AI_ARC_CRL_Rank = AI_ARC_CRL_Rank
        self.AI_ARC_OBC_NCL_Rank = AI_ARC_OBC_NCL_Rank
        self.AI_ARC_SC_Rank = AI_ARC_SC_Rank
        self.AI_ARC_ST_Rank = AI_ARC_ST_Rank
        self.AI_ARC_EWS_Rank = AI_ARC_EWS_Rank
        self.AI_ARC_CRL_PD_Rank = AI_ARC_CRL_PD_Rank
        self.AI_ARC_OBC_NCL_PD_Rank = AI_ARC_OBC_NCL_PD_Rank
        self.AI_ARC_SC_PD_Rank = AI_ARC_SC_PD_Rank
        self.AI_ARC_ST_PD_Rank = AI_ARC_ST_PD_Rank
        self.AI_ARC_EWS_PD_Rank = AI_ARC_EWS_PD_Rank
        
        self.AI_Pln_CRL_Rank = AI_Pln_CRL_Rank
        self.AI_Pln_OBC_NCL_Rank = AI_Pln_OBC_NCL_Rank
        self.AI_Pln_SC_Rank = AI_Pln_SC_Rank
        self.AI_Pln_ST_Rank = AI_Pln_ST_Rank
        self.AI_Pln_EWS_Rank = AI_Pln_EWS_Rank
        self.AI_Pln_CRL_PD_Rank = AI_Pln_CRL_PD_Rank
        self.AI_Pln_OBC_NCL_PD_Rank = AI_Pln_OBC_NCL_PD_Rank
        self.AI_Pln_SC_PD_Rank = AI_Pln_SC_PD_Rank
        self.AI_Pln_ST_PD_Rank = AI_Pln_ST_PD_Rank
        self.AI_Pln_EWS_PD_Rank = AI_Pln_EWS_PD_Rank

def load_candidates(filename):
    df = pd.read_csv(filename)
    filtered_df = df.sort_values('Adv_CRL_Rank')
    filtered_df = filtered_df[filtered_df['Nationality'] != 4]
    return {row['RollNo']: Candidate(row['RollNo']
      ,row['AI_Eng_CRL_Rank']  
      ,row['AI_Eng_OBC_NCL_Rank']
      ,row['AI_Eng_SC_Rank']
      ,row['AI_Eng_ST_Rank']
      ,row['AI_Eng_EWS_Rank']
      ,row['AI_Eng_CRL_PD_Rank']
      ,row['AI_Eng_OBC_NCL_PD_Rank']
      ,row['AI_Eng_SC_PD_Rank']
      ,row['AI_Eng_ST_PD_Rank']
      ,row['AI_Eng_EWS_PD_Rank']
      ,row['AI_ARC_CRL_Rank']
      ,row['AI_ARC_OBC_NCL_Rank']
      ,row['AI_ARC_SC_Rank']
      ,row['AI_ARC_ST_Rank']
      ,row['AI_ARC_EWS_Rank']
      ,row['AI_ARC_CRL_PD_Rank']
      ,row['AI_ARC_OBC_NCL_PD_Rank']
      ,row['AI_ARC_SC_PD_Rank']
      ,row['AI_ARC_ST_PD_Rank']
      ,row['AI_ARC_EWS_PD_Rank']
      ,row['AI_Pln_CRL_Rank']
      ,row['AI_Pln_OBC_NCL_Rank']
      ,row['AI_Pln_SC_Rank']
      ,row['AI_Pln_ST_Rank']
      ,row['AI_Pln_EWS_Rank']
      ,row['AI_Pln_CRL_PD_Rank']
      ,row['AI_Pln_OBC_NCL_PD_Rank']
      ,row['AI_Pln_SC_PD_Rank']
      ,row['AI_Pln_ST_PD_Rank']
      ,row['AI_Pln_EWS_PD_Rank']) for index, row in filtered_df.iterrows()}

def load_programs(filename):
    df = pd.read_csv(filename)
    return {(row['InstCd'], row['BrCd'], row['Cat'], row['SubCat']): Program(row['InstCd'], row['BrCd'], row['StCd1'], row['Cat'], row['SubCat'], 'B', row['TSeat']) for index, row in df.iterrows()}

# Cat,SubCat,Gender,StCd1,StCd2,StCd3,StCd4,StCd5,StCd6,StCd7,StCd8,StCd9,StCd10,TSeat
candidates = load_candidates(path_candidates)




In [43]:
programs = load_programs(path_seats)
print("The easy part is done.")

The easy part is done.


In [44]:
def getRank0(roll_no, program):
    candidate = candidates[roll_no]
    if program.brcode == "4501":
        if program.cat == "OP":
            if program.subcat == "NO":
                return candidate.AI_Pln_CRL_Rank
            else:
                return candidate.AI_Pln_CRL_PD_Rank
        elif program.cat == "BC":
            if program.subcat == "NO":
                return candidate.AI_Pln_OBC_NCL_Rank
            else:
                return candidate.AI_Pln_OBC_NCL_PD_Rank
        elif program.cat == "SC":
            if program.subcat == "NO":
                return candidate.AI_Pln_SC_Rank
            else:
                return candidate.AI_Pln_SC_PD_Rank
        elif program.cat == "EW":
            if program.subcat == "NO":
                return candidate.AI_Pln_EWS_Rank
            else:
                return candidate.AI_Pln_EWS_PD_Rank
        elif program.cat == "ST":
            if program.subcat == "NO":
                return candidate.AI_Pln_ST_Rank
            else:
                return candidate.AI_Pln_ST_PD_Rank  
            
    elif program.brcode == "5101" or program.brcode == "5102" or program.brcode == "5103":
        if program.cat == "OP":
            if program.subcat == "NO":
                return candidate.AI_ARC_CRL_Rank
            else:
                return candidate.AI_ARC_CRL_PD_Rank
        elif program.cat == "BC":
            if program.subcat == "NO":
                return candidate.AI_ARC_OBC_NCL_Rank
            else:
                return candidate.AI_ARC_OBC_NCL_PD_Rank
        elif program.cat == "SC":
            if program.subcat == "NO":
                return candidate.AI_ARC_SC_Rank
            else:
                return candidate.AI_ARC_SC_PD_Rank
        elif program.cat == "EW":
            if program.subcat == "NO":
                return candidate.AI_ARC_EWS_Rank
            else:
                return candidate.AI_ARC_EWS_PD_Rank
        elif program.cat == "ST":
            if program.subcat == "NO":
                return candidate.AI_ARC_ST_Rank
            else:
                return candidate.AI_ARC_ST_PD_Rank
    else:
        if program.cat == "OP":
            if program.subcat == "NO":
                return candidate.AI_Eng_CRL_Rank
            else:
                return candidate.AI_Eng_CRL_PD_Rank
        elif program.cat == "BC":
            if program.subcat == "NO":
                return candidate.AI_Eng_OBC_NCL_Rank
            else:
                return candidate.AI_Eng_OBC_NCL_PD_Rank
        elif program.cat == "SC":
            if program.subcat == "NO":
                return candidate.AI_Eng_SC_Rank
            else:
                return candidate.AI_Eng_SC_PD_Rank
        elif program.cat == "EW":
            if program.subcat == "NO":
                return candidate.AI_Eng_EWS_Rank
            else:
                return candidate.AI_Eng_EWS_PD_Rank
        elif program.cat == "ST":
            if program.subcat == "NO":
                return candidate.AI_Eng_ST_Rank
            else:
                return candidate.AI_Eng_ST_PD_Rank
        else:
            return -1

In [45]:
def process_chunk(chunk):
    local_candidates = {}
    for index, row in chunk.iterrows():
        roll_no = int(row['RollNo'])
        choice_no = row['ChoiceNo']
        v_choice_no = int(row['ChoiceNo_V'])
        inst_code = row['InstCode']
        brcode = row['Brcode']
        cat = row['Cat_V']
        subcat = row['SubCat_V']
        if roll_no in candidates.keys():
            if roll_no not in local_candidates:
                local_candidates[roll_no] = {'choice_list': {}, 'choice_list_mapper': {}, 'max_index': 0}
            program_key = (inst_code, brcode, cat, subcat)
            program = programs.get(program_key)
            local_candidates[roll_no]['choice_list'][v_choice_no - 1] = program
            local_candidates[roll_no]['choice_list_mapper'][v_choice_no - 1] = choice_no
            local_candidates[roll_no]['max_index'] += 1
    return local_candidates

def merge_results(results):
    for result in results:
        for roll_no, data in result.items():
            if roll_no in candidates:
                candidates[roll_no].choice_list.update(data['choice_list'])
                candidates[roll_no].choice_list_mapper.update(data['choice_list_mapper'])
                candidates[roll_no].max_index += data['max_index']

def load_choices_parallel(filename, chunk_size=500000):
    df = pd.read_csv(filename, chunksize=chunk_size)
    futures = []
    with ProcessPoolExecutor(max_workers=20) as executor:
        for chunk in df:
            futures.append(executor.submit(process_chunk, chunk))
        results = []
        for future in as_completed(futures):
            results.append(future.result())
    merge_results(results)

    
start_time = time.time()
load_choices_parallel(path_normChoices)     
end_time = time.time()  # Record end time
elapsed_time = end_time - start_time  # Calculate elapsed time
print(f"Time taken for loading choices in parallel: {elapsed_time:.2f} seconds")
print("The tough part is done.")


Time taken for loading choices in parallel: 0.20 seconds
The tough part is done.


In [46]:
def resetter():
    for program_key, _ in programs.items():
        programs[program_key].waitlist=[]
    for candidate_key, _ in candidates.items():
        candidates[candidate_key].current_index=1
    print("Successfully reset.")
resetter()
allocations={}
ds_inst=defaultdict(list)

Successfully reset.


In [47]:
def spl_printer(roll_no):
    candidate=candidates[roll_no]
    print(f"Roll No: {candidate.roll_no}")
    print(f"  Choices: {candidate.max_index}")
    print(f"  Current Index: {candidate.current_index}")
    print(f"  Choice List:")
    for i in range(0, candidate.max_index):
        if candidate.choice_list[i]:
            print(f"    {i+1}) {candidate.choice_list_mapper[i]} {candidate.choice_list[i].inst_code},{candidate.choice_list[i].brcode},{candidate.choice_list[i].quota},{candidate.choice_list[i].cat},{candidate.choice_list[i].subcat},{candidate.choice_list[i].gender} [{getRank0(candidate.roll_no, candidate.choice_list[i])}]")
    if candidate.roll_no in allocations.keys():
        cAlloc = allocations[candidate.roll_no]
        if cAlloc:
            print(f"  Current Allotment : {cAlloc}")
            print(f"    {cAlloc.inst_code},{cAlloc.brcode},{cAlloc.quota},{cAlloc.cat},{cAlloc.subcat},{cAlloc.gender} [{getRank0(candidate.roll_no, cAlloc)}]")

    print()

In [48]:
spl_printer(240310002160)

Roll No: 240310002160
  Choices: 8
  Current Index: 1
  Choice List:
    1) 1 208,4110,AN,OP,NO,B [349604.0]
    2) 2 208,4114,AN,OP,NO,B [349604.0]
    3) 3 208,4111,AN,OP,NO,B [349604.0]
    4) 4 208,4109,AN,OP,NO,B [349604.0]
    5) 5 208,411F,AN,OP,NO,B [349604.0]
    6) 6 208,4125,AN,OP,NO,B [349604.0]
    7) 7 208,4127,AN,OP,NO,B [349604.0]
    8) 8 208,4107,AN,OP,NO,B [349604.0]



In [49]:
# sups={}
# totSeats={}
# for program_key, program in programs.items():
#     if program.capacity<len(program.waitlist):
#         print(f"Supernumerary in {program_key[0]}, {program_key[1]}, {program_key[2]}, {program_key[3]}, {program_key[4]}, {program_key[5]} : {len(program.waitlist)-program.capacity} seats")
#         if program_key[3] not in sups.keys():
#             sups[program_key[3]]=0
#         sups[program_key[3]]+=len(program.waitlist)-program.capacity
#         if program_key[3]+program_key[4] not in totSeats.keys():
#             totSeats[program_key[3]+program_key[4]]=0
#         totSeats[program_key[3]+program_key[4]]+=len(program.waitlist)
#     else:
#         if program_key[3]+program_key[4] not in totSeats.keys():
#             totSeats[program_key[3]+program_key[4]]=0
#         totSeats[program_key[3]+program_key[4]]+=program.capacity
               
# print(sups)
# print(totSeats)

In [50]:
start_time = time.time()

def getRank(roll_no, program):
    candidate = candidates[roll_no]
    if program.brcode == "4501":
        if program.cat == "OP":
            if program.subcat == "NO":
                return candidate.AI_Pln_CRL_Rank
            else:
                return candidate.AI_Pln_CRL_PD_Rank
        elif program.cat == "BC":
            if program.subcat == "NO":
                return candidate.AI_Pln_OBC_NCL_Rank
            else:
                return candidate.AI_Pln_OBC_NCL_PD_Rank
        elif program.cat == "SC":
            if program.subcat == "NO":
                return candidate.AI_Pln_SC_Rank
            else:
                return candidate.AI_Pln_SC_PD_Rank
        elif program.cat == "EW":
            if program.subcat == "NO":
                return candidate.AI_Pln_EWS_Rank
            else:
                return candidate.AI_Pln_EWS_PD_Rank
        elif program.cat == "ST":
            if program.subcat == "NO":
                return candidate.AI_Pln_ST_Rank
            else:
                return candidate.AI_Pln_ST_PD_Rank  
            
    elif program.brcode == "5101" or program.brcode == "5102" or program.brcode == "5103":
        if program.cat == "OP":
            if program.subcat == "NO":
                return candidate.AI_ARC_CRL_Rank
            else:
                return candidate.AI_ARC_CRL_PD_Rank
        elif program.cat == "BC":
            if program.subcat == "NO":
                return candidate.AI_ARC_OBC_NCL_Rank
            else:
                return candidate.AI_ARC_OBC_NCL_PD_Rank
        elif program.cat == "SC":
            if program.subcat == "NO":
                return candidate.AI_ARC_SC_Rank
            else:
                return candidate.AI_ARC_SC_PD_Rank
        elif program.cat == "EW":
            if program.subcat == "NO":
                return candidate.AI_ARC_EWS_Rank
            else:
                return candidate.AI_ARC_EWS_PD_Rank
        elif program.cat == "ST":
            if program.subcat == "NO":
                return candidate.AI_ARC_ST_Rank
            else:
                return candidate.AI_ARC_ST_PD_Rank
    else:
        if program.cat == "OP":
            if program.subcat == "NO":
                return candidate.AI_Eng_CRL_Rank
            else:
                return candidate.AI_Eng_CRL_PD_Rank
        elif program.cat == "BC":
            if program.subcat == "NO":
                return candidate.AI_Eng_OBC_NCL_Rank
            else:
                return candidate.AI_Eng_OBC_NCL_PD_Rank
        elif program.cat == "SC":
            if program.subcat == "NO":
                return candidate.AI_Eng_SC_Rank
            else:
                return candidate.AI_Eng_SC_PD_Rank
        elif program.cat == "EW":
            if program.subcat == "NO":
                return candidate.AI_Eng_EWS_Rank
            else:
                return candidate.AI_Eng_EWS_PD_Rank
        elif program.cat == "ST":
            if program.subcat == "NO":
                return candidate.AI_Eng_ST_Rank
            else:
                return candidate.AI_Eng_ST_PD_Rank
        else:
            return -1

def process_candidates():
    start_time = time.time()
    allocations = {}
    while stack:
        candidate = stack.pop()
        if candidate.current_index<=candidate.max_index:
            preference = candidates[candidate.roll_no].choice_list[candidate.current_index-1]
            if preference :                
                if candidate.roll_no==240310017480:
                    print(f"{preference.inst_code}, {preference.brcode}, {preference.cat}, {preference.subcat}")                    
                preference = programs[preference.inst_code, preference.brcode, preference.cat, preference.subcat]
                rk = getRank(candidate.roll_no, preference)
                if preference.add_candidate(rk, candidate.roll_no):
                    allocations[candidate.roll_no] = preference
                    if candidate.roll_no==240310017480:
                        print(f"Allotted {preference.inst_code}, {preference.brcode}, {preference.cat}, {preference.subcat} {preference.capacity-len(preference.waitlist)} {preference.waitlist}")               
                    if len(preference.waitlist) > preference.capacity:
                        _, roll_no = heapq.heappop(preference.waitlist)                        
                        if 240310017480==roll_no:
                            print(f"Removed {preference.inst_code}, {preference.brcode}, {preference.cat}, {preference.subcat} {preference.capacity-len(preference.waitlist)} {preference.waitlist}")  
                        candidates[roll_no].current_index += 1
                        candidate = candidates[roll_no] 
                        allocations[roll_no] = None
                        stack.append(candidate)
                        # if last_rank > preference.min_cutoff:
                        #     count_same_rank = sum(1 for x in preference.waitlist if -x[0] == last_rank)
                        #     if len(preference.waitlist) - count_same_rank >= preference.capacity:
                        #         removed_candidates = []
                        #         while preference.waitlist and -preference.waitlist[0][0] == last_rank:
                        #             _, roll_no = heapq.heappop(preference.waitlist)
                        #             removed_candidates.append(roll_no)
                        #         for roll_no in removed_candidates:
                        #             if 240310017480==roll_no:
                        #                 print(f"Removed {preference.inst_code}, {preference.brcode}, {preference.cat}, {preference.subcat} {preference.waitlist}")  
                        #             candidates[roll_no].current_index += 1
                        #             candidate = candidates[roll_no] 
                        #             allocations[roll_no] = None
                        #             stack.append(candidate)
                        #     # else:
                        #     #     print(f"At {preference.inst_code}, {preference.brcode}, {preference.quota}, {preference.cat}, {preference.subcat}, {preference.gender}, same rank {rk} - Extra {len(preference.waitlist) - count_same_rank - preference.capacity}")
                        #         # programs[preference.inst_code, preference.brcode, preference.quota, preference.cat, preference.subcat, preference.gender, 0].capacity+=1
                else:
                    candidates[candidate.roll_no].current_index += 1
                    stack.append(candidate)
                
    print(f"Time taken: {time.time() - start_time:.2f} seconds")
    return allocations, programs

stack = []
filtered_candidates = [c for c in candidates.values() if c.AI_Eng_CRL_Rank > 0]
filtered_candidates.sort(key=lambda c: (c.AI_Eng_CRL_Rank))
filtered_candidates1 = [c for c in candidates.values() if c.AI_Eng_CRL_Rank == 0]
stack.extend(filtered_candidates1)
stack.extend(filtered_candidates[::-1])
allocations, programs = process_candidates()
end_time = time.time()  # Record end time
elapsed_time = end_time - start_time  # Calculate elapsed time
print(f"Time taken for allocation: {elapsed_time:.2f} seconds")


229, 4110, OP, NO
Allotted 229, 4110, OP, NO -1 [(-518127.0, 240310017480)]
Removed 229, 4110, OP, NO 0 []
229, 4110, EW, NO
Allotted 229, 4110, EW, NO -1 [(-76144.0, 240310017480)]
Removed 229, 4110, EW, NO 0 []
229, 4125, OP, NO
Allotted 229, 4125, OP, NO -1 [(-518127.0, 240310017480), (-174199.0, 240311019517)]
Removed 229, 4125, OP, NO 0 [(-174199.0, 240311019517)]
229, 4125, EW, NO
Allotted 229, 4125, EW, NO -1 [(-76144.0, 240310017480)]
Removed 229, 4125, EW, NO 0 []
229, 4114, OP, NO
Allotted 229, 4114, OP, NO -1 [(-518127.0, 240310017480), (-40779.0, 240310111770)]
Removed 229, 4114, OP, NO 0 [(-40779.0, 240310111770)]
229, 4114, EW, NO
Allotted 229, 4114, EW, NO -1 [(-76144.0, 240310017480)]
Removed 229, 4114, EW, NO 0 []
229, 4109, OP, NO
Allotted 229, 4109, OP, NO -1 [(-518127.0, 240310017480), (-326820.0, 240310654405), (-183888.0, 240310584249), (-101330.0, 240310766344), (-219422.0, 240310668865)]
Removed 229, 4109, OP, NO 0 [(-326820.0, 240310654405), (-219422.0, 2403106

In [51]:
old_pgm=programs
for program_key, program in programs.items():
    if program_key[3]=="PH" and program.capacity>len(program.waitlist):
        print(programs[program_key[0], program_key[1], program_key[2], "NO"].capacity)
        programs[program_key[0], program_key[1], program_key[2], "NO"].capacity+=program.capacity-len(program.waitlist)
        print(programs[program_key[0], program_key[1], program_key[2], "NO"].capacity)
        print(f"Shifted {program.capacity-len(program.waitlist)} from {program_key[0]}, {program_key[1]}, {program_key[2]}, {"PH"} to {program_key[0]}, {program_key[1]}, {program_key[2]}, {"NO"}")
        programs[program_key].capacity=len(program.waitlist)
        

1
2
Shifted 1 from 206, 4114, OP, PH to 206, 4114, OP, NO
3
4
Shifted 1 from 208, 4125, OP, PH to 208, 4125, OP, NO
4
5
Shifted 1 from 229, 4109, OP, PH to 229, 4109, OP, NO


In [52]:
def ENDresetter():
    for program_key, _ in programs.items():
        programs[program_key].waitlist=[]
    for candidate_key, _ in candidates.items():
        candidates[candidate_key].current_index=1
    print("Successfully reset.")
ENDresetter()
allocations={}
filtered_candidates = [c for c in candidates.values() if c.AI_Eng_CRL_Rank > 0]
filtered_candidates.sort(key=lambda c: (c.AI_Eng_CRL_Rank))
filtered_candidates1 = [c for c in candidates.values() if c.AI_Eng_CRL_Rank == 0]
stack.extend(filtered_candidates1)
stack.extend(filtered_candidates[::-1])
allocations, programs = process_candidates()        

Successfully reset.
229, 4110, OP, NO
Allotted 229, 4110, OP, NO -1 [(-518127.0, 240310017480)]
Removed 229, 4110, OP, NO 0 []
229, 4110, EW, NO
Allotted 229, 4110, EW, NO -1 [(-76144.0, 240310017480)]
Removed 229, 4110, EW, NO 0 []
229, 4125, OP, NO
Allotted 229, 4125, OP, NO -1 [(-518127.0, 240310017480), (-174199.0, 240311019517)]
Removed 229, 4125, OP, NO 0 [(-174199.0, 240311019517)]
229, 4125, EW, NO
Allotted 229, 4125, EW, NO -1 [(-76144.0, 240310017480)]
Removed 229, 4125, EW, NO 0 []
229, 4114, OP, NO
Allotted 229, 4114, OP, NO -1 [(-518127.0, 240310017480), (-40779.0, 240310111770)]
Removed 229, 4114, OP, NO 0 [(-40779.0, 240310111770)]
229, 4114, EW, NO
Allotted 229, 4114, EW, NO -1 [(-76144.0, 240310017480)]
Removed 229, 4114, EW, NO 0 []
229, 4109, OP, NO
Allotted 229, 4109, OP, NO 0 [(-518127.0, 240310017480), (-326820.0, 240310654405), (-183888.0, 240310584249), (-101330.0, 240310766344), (-219422.0, 240310668865)]
Time taken: 0.00 seconds


In [53]:
# sum_ds=0
# for inst_code, students in ds_inst.items():
#     print(f"Institution Code: {inst_code}")
#     sum_ds+=len(students)
#     for student in students:
#         print(f"  Roll No: {student['roll_no']}, Branch Code: {student['brcode']}, Rank: {student['rank']}")
# print(sum_ds)

In [54]:
print("Final Allocations:")
for roll_no, preference in allocations.items():
    candidate = candidates.get(roll_no)
    if preference:
        print(f"Candidate {roll_no} is allocated to program {preference.inst_code}'s {preference.brcode} with {preference.quota}, {preference.cat}, {preference.subcat}, {preference.gender} : Mains-{candidate.AI_Eng_CRL_Rank}  Choice - {candidate.current_index}")
  
print("\nFinal Waitlists:")
for program_key, program in programs.items():
    if len(program.waitlist)>0:
        print(f"Program {program_key} with {program.capacity} waitlist: {program.waitlist} with {program.capacity - len(program.waitlist)} remaining seats")
    
print("\nFinal Cutoffs:")
for program_key, program in programs.items():
    if len(program.waitlist)>0:
        last_rank = -program.waitlist[0][0]
        print(f"Program {program_key} cutoff: {last_rank}")

Final Allocations:
Candidate 240310111770 is allocated to program 229's 4114 with DD, OP, NO, B : Mains-40779.0  Choice - 2
Candidate 240310106087 is allocated to program 208's 4110 with AN, OP, NO, B : Mains-42505.0  Choice - 1
Candidate 240310539947 is allocated to program 208's 4110 with AN, OP, NO, B : Mains-43125.0  Choice - 1
Candidate 240311177663 is allocated to program 208's 4110 with AN, OP, NO, B : Mains-48341.0  Choice - 1
Candidate 240310699622 is allocated to program 229's 4110 with DD, BC, NO, B : Mains-51750.0  Choice - 2
Candidate 240310319469 is allocated to program 208's 4110 with AN, OP, NO, B : Mains-95839.0  Choice - 1
Candidate 240310766344 is allocated to program 229's 4109 with DD, OP, NO, B : Mains-101330.0  Choice - 1
Candidate 240311157706 is allocated to program 229's 4107 with DD, OP, NO, B : Mains-111349.0  Choice - 3
Candidate 240310442155 is allocated to program 208's 4111 with AN, OP, NO, B : Mains-125062.0  Choice - 1
Candidate 240310322947 is allocat

In [55]:
import csv

unallocated = []

def getPRank(roll_no, program):
    candidate = candidates[roll_no]
    if program.cat == "OP":
        return str(candidate.Adv_Prep_CRL_PD_Rank)+"P"
    elif program.cat == "BC":
        return str(candidate.Adv_Prep_OBC_NCL_PD_Rank)+"P"
    elif program.cat == "EW":
        return str(candidate.Adv_Prep_EWS_PD_Rank)+"P"
    elif program.cat == "SC":
        if program.subcat == "NO":
            return str(candidate.Adv_Prep_SC_Rank)+"P"
        else:
            return str(candidate.Adv_Prep_SC_PD_Rank)+"P"
    elif program.cat == "ST":
        if program.subcat == "NO":
            return str(candidate.Adv_Prep_ST_Rank)+"P"
        else:
            return str(candidate.Adv_Prep_ST_PD_Rank)+"P"
    else:
        return -1

with open(path_output, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Roll No", "Program Code", "Branch Code", "Quota", "Category", "Subcategory", "Gender", "Rank", "Choice Index", "Flag"])
    for roll_no, preference in allocations.items():
        candidate = candidates.get(roll_no)
        if candidate.current_index>candidate.max_index:
            unallocated.append(candidate.roll_no)
        else:
            isPrep="N"
            if preference:
                allocRk=getRank(roll_no, preference)
                writer.writerow([
                    roll_no,
                    preference.inst_code,
                    preference.brcode,
                    preference.quota,
                    preference.cat,
                    preference.subcat,
                    preference.gender,
                    allocRk,
                    candidate.choice_list_mapper[candidate.current_index-1]
                ])

print("Allocations generated")

with open(path_error, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Roll No"])
    for roll_no in unallocated:
        writer.writerow([roll_no])

print("Unallocations generated")

Allocations generated
Unallocations generated


In [56]:
# sups={}
# totSeats={}
# alloSeats={}
# for program_key, program in programs.items():
#     if program_key[3]+program_key[4] not in alloSeats.keys():
#         alloSeats[program_key[3]+program_key[4]]=0
#     alloSeats[program_key[3]+program_key[4]]+=len(program.waitlist)
#     if program.capacity<len(program.waitlist):
#         print(f"Supernumerary in {program_key[0]}, {program_key[1]}, {program_key[2]}, {program_key[3]}, {program_key[4]}, {program_key[5]} : {len(program.waitlist)-program.capacity} seats")
#         if program_key[3] not in sups.keys():
#             sups[program_key[3]]=0
#         sups[program_key[3]]+=len(program.waitlist)-program.capacity
#         if program_key[3]+program_key[4] not in totSeats.keys():
#             totSeats[program_key[3]+program_key[4]]=0
#         totSeats[program_key[3]+program_key[4]]+=len(program.waitlist)
#     else:
#         if program_key[3]+program_key[4] not in totSeats.keys():
#             totSeats[program_key[3]+program_key[4]]=0
#         totSeats[program_key[3]+program_key[4]]+=program.capacity
        
        
# print(sups)
# print(alloSeats)
# print(totSeats)

In [57]:
def load_allotment(filename):
    df = pd.read_csv(filename)
    return {(row['Rollno'], row['Optno']): programs[row['Instcd'], row['Brcd'], row['AllottedCat'][:2], row['AllottedCat'][2:]] for index, row in df.iterrows()}
# RoundNo,Rollno,Birth_Cat,Rank,Optno,Instcd,Brcd,AllottedCat,AllottedQuota,AllottedGender,Flag,SupNumReason,WithDraw,RStatus,SStatus,Status,ADate,PIStatus

   # return {(row['RollNO'], row['ChoiceNo']): programs[row['InstCode'], row['Brcode'],  row['AllottedQuota'], row['AllottedCategory'][:2], row['AllottedCategory'][2:], row['AllottedGender'], 0] for index, row in df.iterrows()}
given_all = load_allotment(path_allotment)

for keyo, alloted in given_all.items():    
    roll_no=keyo[0]
    chno=keyo[1]
    if roll_no in candidates.keys():
        if  roll_no in allocations:
            if allocations[roll_no]!=alloted:
                if allocations[roll_no]:
                    print(f"N {roll_no} was alloted ({chno})({alloted.inst_code},{alloted.brcode},{alloted.quota},{alloted.cat}{alloted.subcat},{alloted.gender}) but got ({candidates[roll_no].choice_list_mapper[candidates[roll_no].current_index-1]})({allocations[roll_no].inst_code},{allocations[roll_no].brcode},{allocations[roll_no].quota},{allocations[roll_no].cat}{allocations[roll_no].subcat},{allocations[roll_no].gender}) with {candidates[roll_no].AI_Eng_CRL_Rank}")
                else:
                    print(f"N {roll_no} was alloted ({chno})({alloted.inst_code},{alloted.brcode},{alloted.quota},{alloted.cat}{alloted.subcat},{alloted.gender}) with {candidates[roll_no].AI_Eng_CRL_Rank}")
        else:
            print(f"N {roll_no} was alloted ({chno})({alloted.inst_code},{alloted.brcode},{alloted.quota},{alloted.cat}{alloted.subcat},{alloted.gender}) but didn't enter")
    # elif not roll_no in foreignCandidates.keys():    
    #     if candidates[roll_no].choice_list_mapper[candidates[roll_no].current_index-1]!=chno:
    #                 print(f"DS {roll_no} was alloted ({chno})({alloted.inst_code},{alloted.brcode},{alloted.quota},{alloted.cat}{alloted.subcat},{alloted.gender}) but got ({candidates[roll_no].choice_list_mapper[candidates[roll_no].current_index-1]})({allocations[roll_no].inst_code},{allocations[roll_no].brcode},{allocations[roll_no].quota},{allocations[roll_no].cat}{allocations[roll_no].subcat},{allocations[roll_no].gender}) with {candidates[roll_no].AI_Eng_CRL_Rank} and {candidates[roll_no].Adv_CRL_Rank}")

In [58]:
spl_printer(240310002160)

Roll No: 240310002160
  Choices: 8
  Current Index: 7
  Choice List:
    1) 1 208,4110,AN,OP,NO,B [349604.0]
    2) 2 208,4114,AN,OP,NO,B [349604.0]
    3) 3 208,4111,AN,OP,NO,B [349604.0]
    4) 4 208,4109,AN,OP,NO,B [349604.0]
    5) 5 208,411F,AN,OP,NO,B [349604.0]
    6) 6 208,4125,AN,OP,NO,B [349604.0]
    7) 7 208,4127,AN,OP,NO,B [349604.0]
    8) 8 208,4107,AN,OP,NO,B [349604.0]
  Current Allotment : <__main__.Program object at 0x70d085efab10>
    208,4127,AN,OP,NO,B [349604.0]



In [59]:
spl_printer(240310017480 )

Roll No: 240310017480
  Choices: 10
  Current Index: 7
  Choice List:
    1) 1 229,4110,DD,OP,NO,B [518127.0]
    2) 1 229,4110,DD,EW,NO,B [76144.0]
    3) 2 229,4125,DD,OP,NO,B [518127.0]
    4) 2 229,4125,DD,EW,NO,B [76144.0]
    5) 3 229,4114,DD,OP,NO,B [518127.0]
    6) 3 229,4114,DD,EW,NO,B [76144.0]
    7) 4 229,4109,DD,OP,NO,B [518127.0]
    8) 4 229,4109,DD,EW,NO,B [76144.0]
    9) 5 229,4107,DD,OP,NO,B [518127.0]
    10) 5 229,4107,DD,EW,NO,B [76144.0]
  Current Allotment : <__main__.Program object at 0x70d085efb020>
    229,4109,DD,OP,NO,B [518127.0]

